In [17]:
import mediapipe as mp
import math
import cv2
import pyautogui

In [3]:
# Setup the mediapipe model
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.3)
mp_drawing = mp.solutions.drawing_utils

In [14]:
def detectPose(image, pose, display=True):
    """Places landmarks on an image."""
    output_image = image.copy()
    
    # Identify all landmarks
    imageRGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(imageRGB)
    
    # Retrieve the height and width of the input image.
    height, width, _ = image.shape
    
    # Append all landmarks and draw them on the OUTPUT image
    landmarks = []

    if results.pose_landmarks:
        mp_drawing.draw_landmarks(image=output_image, landmark_list=results.pose_landmarks,
                                  connections=mp_pose.POSE_CONNECTIONS)

        for landmark in results.pose_landmarks.landmark:
            landmarks.append((int(landmark.x * width), int(landmark.y * height),
                                  (landmark.z * width)))
    
    return output_image, landmarks

In [15]:
def calculateAngle(landmark1, landmark2, landmark3):
    """Calculate the angle between 3 landmarks."""
    # Get the required landmarks coordinates.
    x1, y1, _ = landmark1
    x2, y2, _ = landmark2
    x3, y3, _ = landmark3
 
    # Calculate the angle between the three points
    angle = math.degrees(math.atan2(y3 - y2, x3 - x2) - math.atan2(y1 - y2, x1 - x2))
    
    # Make the angle positive (FOR NOW)
    if angle < 0:
        angle += 360

    return angle

In [19]:
def check_salute(shoulder, elbow, wrist, frame):
    """Checks to see if the person is saluting."""
    is_saluting = True if 35 < calculateAngle(shoulder, elbow, wrist) < 50 else False
    if is_saluting:
        cv2.putText(frame, "Saluting!", (10, 30), cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 2)
    else:
        cv2.putText(frame, "nope", (10, 30), cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 2)
    return is_saluting


In [37]:
def move_forward():
    """Press the w key on the user's laptop."""
    pyautogui.keyDown('W')
    pyautogui.keyUp('W')
    

In [38]:
pose_video = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, model_complexity=1)
 
# Initialize the VideoCapture object to read from the webcam.
video = cv2.VideoCapture(0)
cv2.namedWindow('Pose Detection', cv2.WINDOW_NORMAL)

# Initialize a variable to store the time of the previous frame.
time1 = 0
 
# Iterate until the video is accessed successfully.
while video.isOpened():
    
    # Read a frame.
    ok, frame = video.read()
    
    # Check if frame is not read properly.
    if not ok:
        break
    
    frame = cv2.flip(frame, 1)
    
    # Adjust the frame
    frame_height, frame_width, _ =  frame.shape
    frame = cv2.resize(frame, (int(frame_width * (640 / frame_height)), 640))
    
    # Perform Pose landmark detection.
    frame, landmarks = detectPose(frame, pose_video, display=False)

    if landmarks:
        is_saluting = check_salute(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value], landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value], landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value], frame)
        if is_saluting:
            move_forward()
    
    # Display the frame.
    cv2.imshow('Pose Detection', frame)
    
    # Check to see if the user pressed 'esc'
    k = cv2.waitKey(1) & 0xFF
    if(k == 27):
        break
 
# Clean Up
video.release()
cv2.destroyAllWindows()